In [1]:
# 导入需要的库
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict as ddict


In [2]:
with open("session/map/RoadLine.json","r",encoding="utf-8") as f:
    old_roadlines = json.load(f)
old_roadlines

[{'type': 'Feature',
  'properties': {'osm_id': 0,
   'NAME': '无名路',
   'EC': 'null',
   'TYPE': '0630',
   'WIDE': '3.3',
   'SPIDE': '30km/h',
   'LENGTH': '4126.45504185466',
   'RTYPE': '无名道路',
   'LANE': 1.0,
   'Shape_Leng': 4126.44842905,
   'IsBreak': 'null',
   'IsThreaten': 'null',
   'IsBlocked': 'null'},
  'geometry': {'type': 'LineString',
   'coordinates': [[105.7183173, 38.862001],
    [105.7183259, 38.8619021],
    [105.7184521, 38.8607224],
    [105.7186006, 38.8588208],
    [105.7187395, 38.857101],
    [105.7188044, 38.8562526],
    [105.7188843, 38.8552084],
    [105.71892594512019, 38.85427076107701],
    [105.7190387, 38.8535152],
    [105.7190765, 38.8529761],
    [105.7191653, 38.8520167],
    [105.7192096, 38.8515383],
    [105.7192119, 38.851476],
    [105.7193286, 38.8500361],
    [105.7193474, 38.8493068],
    [105.7193759, 38.848847],
    [105.719511, 38.8470808],
    [105.7195636, 38.8464276],
    [105.7196342, 38.8453099],
    [105.7197531, 38.8438762],
 

In [3]:
len(old_roadlines)

377

(1)拼接osm_id相同的两条路

In [4]:
roads = []
exists_ids = [] # 记录已经遍历过的osm_id，之后再次遍历到列表内的osm_id则跳过
for i in range(len(old_roadlines)):
    idi = old_roadlines[i]['properties']['osm_id'] # 提取该路的osm_id
    if idi in exists_ids:
        continue # 若该osm_id已遍历过，则跳过
    else:
         exists_ids.append(idi) # 若没有遍历过，则录入exists_ids，并进行以下过程
    same_id_roads = [old_roadlines[i]]
    for j in range(i+1, len(old_roadlines)):
        # 遍历找出所有同osm_id的路，并放到same_id_roads列表中
        idj = old_roadlines[j]['properties']['osm_id']
        if idj == idi:
            same_id_roads.append(old_roadlines[j])
    road = same_id_roads[0]
    for j in range(1,len(same_id_roads)):
        # 将所有重合路径都拼合到一个路径中
        road['geometry']['coordinates'] += same_id_roads[j]['geometry']['coordinates']
    roads.append(road)
new_roadlines = roads
print(f"道路拼接完毕，一共{len(new_roadlines)}条道路")


道路拼接完毕，一共346条道路


(2)找出交叉点

In [5]:
def get_repetpoint(list1, list2):
    # 提取两列表交集（结果不重复）
    repetpoint = []
    for i in list1:
        for j in list2:
            if i == j:
                repetpoint.append(i)
    return repetpoint

In [6]:
roadpoints = []
for i in range(len(new_roadlines)):
    for j in range(i+1,len(new_roadlines)):
        # 找到两条路的重合点，即points1和points2的交集
        points1 = new_roadlines[i]['geometry']['coordinates']
        points2 = new_roadlines[j]['geometry']['coordinates']
        repetpoint = get_repetpoint(points1, points2)
        for p in repetpoint:
            # 将交集中未出现的交叉点录入
            if p not in roadpoints:
                roadpoints.append(p)
print(f"交叉点找寻完毕，共有{len(roadpoints)}个交叉点")
np.save(r'generate_data/roadpoints.npy', roadpoints)
roadpoints

交叉点找寻完毕，共有607个交叉点


[[105.7201544, 38.838968],
 [105.7191653, 38.8520167],
 [105.71892594512019, 38.85427076107701],
 [105.7188044, 38.8562526],
 [105.7195636, 38.8464276],
 [105.7204455, 38.8355347],
 [105.5551353, 38.7631801],
 [105.55381471104553, 38.75339471003323],
 [105.7183173, 38.862001],
 [105.7206857, 38.8322908],
 [105.55212886716531, 38.72573941621608],
 [105.55383211762555, 38.74600924891015],
 [105.55278901872671, 38.734794456845755],
 [105.55257875238914, 38.73232830172084],
 [105.55317779291616, 38.739380737628466],
 [105.6897909, 38.8507837],
 [105.663865183319, 38.847369920986466],
 [105.67372509005864, 38.84971032434993],
 [105.68351590718768, 38.85077741449088],
 [105.7119489, 38.8385664],
 [105.7155551, 38.8387547],
 [105.7255749, 38.8392209],
 [105.6981315, 38.8379045],
 [105.6891193, 38.8374453],
 [105.705115, 38.8382498],
 [105.6338791, 38.8652449],
 [105.6404904, 38.863114],
 [105.6602493, 38.8576063],
 [105.6594438, 38.8612827],
 [105.635401, 38.8525354],
 [105.6193686, 38.861642

（3）邻接矩阵

In [7]:
roadlines1  = {}
for no,d in enumerate(new_roadlines):
    roadlines1[f"{d['properties']['osm_id']}"] = d['geometry']['coordinates']
new_roadlines1=roadlines1
new_roadlines1

{'0': [[105.7183173, 38.862001],
  [105.7183259, 38.8619021],
  [105.7184521, 38.8607224],
  [105.7186006, 38.8588208],
  [105.7187395, 38.857101],
  [105.7188044, 38.8562526],
  [105.7188843, 38.8552084],
  [105.71892594512019, 38.85427076107701],
  [105.7190387, 38.8535152],
  [105.7190765, 38.8529761],
  [105.7191653, 38.8520167],
  [105.7192096, 38.8515383],
  [105.7192119, 38.851476],
  [105.7193286, 38.8500361],
  [105.7193474, 38.8493068],
  [105.7193759, 38.848847],
  [105.719511, 38.8470808],
  [105.7195636, 38.8464276],
  [105.7196342, 38.8453099],
  [105.7197531, 38.8438762],
  [105.7199112, 38.8421801],
  [105.7200146, 38.8409649],
  [105.7201544, 38.838968],
  [105.7203035, 38.8372621],
  [105.7204455, 38.8355347],
  [105.7205187, 38.8348029],
  [105.7206261, 38.8333633],
  [105.7206719, 38.8323907],
  [105.7206857, 38.8322908],
  [105.7207773, 38.8315506],
  [105.55212886716531, 38.72573941621608],
  [105.55237081804714, 38.7276138125836],
  [105.5523091253922, 38.7290944

In [8]:
# 保留每条路径的交叉点
simp_roads = [[] for _ in range(len(new_roadlines1))]
count = ddict(int)
for no,road in enumerate(new_roadlines1.values()):
    for p in road:
        if p in roadpoints:
            count[tuple(p)] += 1
    retained_points = [p in roadpoints for p in road] #【retained_points包含原始所有点，是交叉点返回Ture,否返回False】
    for noo,t in enumerate(retained_points):
        if t:
            simp_roads[no].append(road[noo])
simp_roads

[[[105.7183173, 38.862001],
  [105.7188044, 38.8562526],
  [105.71892594512019, 38.85427076107701],
  [105.7191653, 38.8520167],
  [105.7195636, 38.8464276],
  [105.7201544, 38.838968],
  [105.7204455, 38.8355347],
  [105.7206857, 38.8322908],
  [105.55212886716531, 38.72573941621608],
  [105.55257875238914, 38.73232830172084],
  [105.55278901872671, 38.734794456845755],
  [105.55317779291616, 38.739380737628466],
  [105.55383211762555, 38.74600924891015],
  [105.55381471104553, 38.75339471003323],
  [105.5551353, 38.7631801]],
 [[105.663865183319, 38.847369920986466],
  [105.67372509005864, 38.84971032434993],
  [105.68351590718768, 38.85077741449088],
  [105.6897909, 38.8507837]],
 [[105.7255749, 38.8392209],
  [105.7201544, 38.838968],
  [105.7155551, 38.8387547],
  [105.7119489, 38.8385664],
  [105.705115, 38.8382498],
  [105.6981315, 38.8379045],
  [105.6891193, 38.8374453]],
 [[105.6338791, 38.8652449], [105.6404904, 38.863114]],
 [[105.6594438, 38.8612827], [105.6602493, 38.8576

In [9]:
# 邻接矩阵
Adjacency_matrix = np.zeros((len(roadpoints), len(roadpoints)))
for line in simp_roads:
    for no,p in enumerate (line[:-1]):
        index_first = roadpoints.index(p)
        index_second = roadpoints.index(line[no+1])
        Adjacency_matrix[index_first, index_second] = 1
        Adjacency_matrix[index_second, index_first] = Adjacency_matrix[index_first, index_second]
Adjacency_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
np.save("D1.npy", Adjacency_matrix)

（4）累计距离矩阵

In [11]:
from geopy.distance import geodesic
def distance(p1,p2):
    # 计算小点间的欧氏距离
    return geodesic(tuple(p1[::-1]),tuple(p2[::-1])).m

In [12]:
def get_D(roadpoint,roadlines):
    D = np.ones((len(roadpoint),len(roadpoint)))*float('inf')
    # 遍历导出D矩阵
    for i in range(len(roadlines)):
        points = roadlines[i]['geometry']['coordinates']
        x = -1 # x代表该路的第一个交叉点，x为-1表示尚未找到第一个交叉点
        dist = 0
        for j in range(len(points)):
            if x!=-1: # 找到第一个交叉点后开始累计距离
                dist+=distance(points[j-1],points[j])
            if points[j] in roadpoint:  # 判断该点是否为交叉点
                if x == -1: # 若该交叉点为本路第一个交叉点，则把它置为x
                    x = roadpoint.index(points[j])
                else:       # 若该交叉点不为本路第一个交叉点，则把它置为y，并导入x到y的距离
                    y = roadpoint.index(points[j])
                    if dist < D[x,y]:
                        D[x,y] = dist
                        D[y,x] = dist
                    dist = 0 # 每导入一次距离就刷新dist
                    x = y # x重设为当前点，继续寻找下一个交叉点y
    return D

In [13]:
D=get_D(roadpoints,new_roadlines)
D

array([[         inf,          inf,          inf, ...,          inf,
                 inf,          inf],
       [         inf,          inf, 251.16695588, ...,          inf,
                 inf,          inf],
       [         inf, 251.16695588,          inf, ...,          inf,
                 inf,          inf],
       ...,
       [         inf,          inf,          inf, ...,          inf,
        200.0713258 ,          inf],
       [         inf,          inf,          inf, ..., 200.0713258 ,
                 inf,          inf],
       [         inf,          inf,          inf, ...,          inf,
                 inf,          inf]])

In [14]:
np.save("D.npy", D)

In [15]:
# 同一条路上存在两个相同的交叉点
Candidate_pointD = D
for i in range(len(Candidate_pointD)):
        Candidate_pointD[i,i] = float('inf')

In [16]:
np.save("Candidate_pointD.npy", Candidate_pointD)

In [17]:
Candidate_pointD

array([[         inf,          inf,          inf, ...,          inf,
                 inf,          inf],
       [         inf,          inf, 251.16695588, ...,          inf,
                 inf,          inf],
       [         inf, 251.16695588,          inf, ...,          inf,
                 inf,          inf],
       ...,
       [         inf,          inf,          inf, ...,          inf,
        200.0713258 ,          inf],
       [         inf,          inf,          inf, ..., 200.0713258 ,
                 inf,          inf],
       [         inf,          inf,          inf, ...,          inf,
                 inf,          inf]])

In [18]:
CD = Adjacency_matrix
for i in range(len(CD)):
        CD[i,i] = 0

（5）预测

In [19]:
with open('session/session_3/subject_1/client_path_1.json', 'r', encoding='utf-8') as f:
    info_dict = json.load(f)
    people = info_dict["people"]
    place = info_dict["place_points"]
jsonf = open('sample_data/people_paths.json','r')
df0 = json.load(jsonf)
people_paths = df0['paths']
distance = np.load('Candidate_pointD.npy')
rpoints = np.load('generate_data/roadpoints.npy')

In [20]:
# 固定因素1：与最近避难所的直线距离
def min_refuge_dist(point, ends, rpoints):
    mind = float('inf')
    for end in ends:
        d = cal_distance(rpoints[point], rpoints[end])
        if d<mind:
            mind = d
    return mind
# 固定因素2：与避难所的平均直线距离
def mean_refuge_dist(point, ends, rpoints):
    ds = []
    for end in ends:
        ds.append(cal_distance(rpoints[point], rpoints[end]))
    return np.mean(ds)

In [21]:
def cal_distance(point1, point2):
    return ((point1[0]-point2[0])**2+(point1[1]-point2[1])**2)**(1/2)

In [22]:
# 找到每个民众距离最近的交叉点即为起始点
# 找到每个安全区域最近的交叉点即为终点
def refine_data2(place, people, rpoints):
    starts = []
    for i in range(len(people)):
        starts.append(findnearest([people[i]['people_point']['longitude'],people[i]['people_point']['latitude']], rpoints))
    ends = []
    for i in range(len(place)):
        ends.append(findnearest([place[i]['place_point']['longitude'],place[i]['place_point']['latitude']], rpoints))
    return starts, ends

def findnearest(p, rpoints):
    dmin = float('inf')
    idmin = 0
    for i in range(len(rpoints)):
        d = cal_distance(rpoints[i],p)
        if d < dmin:
            dmin = d
            idmin = i
    return idmin

In [23]:
def get_choices(distance):
    # 计算出各点的下点选择
    choices = []
    for i in range(len(distance)):
        choices.append(np.where(distance[i] < float('inf'))[0].tolist())
    return choices

In [24]:
def get_Xy(people_paths, ends, distance, rpoints):
    X = []
    y = []
    choices = get_choices(distance)
    # 对每条路径的每个点所做出的下点选择做训练
    for path in people_paths:
        for i in range(1,len(path)): 
            xi = []
            choice = choices[path[i-1]] # 获取该点的下点候选点
            yi = choice.index(path[i])
            for j in choice: 
                # 计算固定/动态因素值并加入X中
                #,mean_refuge_dist(j, ends, rpoints)
                xi += [min_refuge_dist(j, ends, rpoints),mean_refuge_dist(j, ends, rpoints)]
            while len(xi)<5*2:
                xi += [0, 0]# 候选点数填充至5个（候选表最大为5个）
            X.append(xi)
            y.append(yi)
    return X,y
starts,ends = refine_data2(place, people, rpoints)
X,y = get_Xy(people_paths, ends, distance, rpoints)
np.save('generate_data/dataX.npy',X)
np.save('generate_data/datay.npy',y)

In [25]:
# 分割数据
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)
# 数据缩放预处理
from sklearn.preprocessing import StandardScaler # Normalizer
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
# 选择模型(随机森林)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=12)
#训练模型
rf = rf.fit(X_train, y_train)
# 在测试集上进行预测
y_predict_on_test = rf.predict(X_test)
# 模型评估
from sklearn.metrics import accuracy_score
print('测试集的准确率为：{:.2f}%'.format(100*accuracy_score(y_test, y_predict_on_test)))   

测试集的准确率为：72.93%


In [26]:
def get_Mm(people_paths, ends, distance, rpoints):
    M = []
    m = []
    choices = get_choices(distance)
    # 对每条路径的每个点所做出的下点选择做训练
    for path in people_paths:
        xi = []
        u = path[0]
        v = path[-1]
        xi = [cal_distance(rpoints[u], rpoints[v]),mean_refuge_dist(u, ends, rpoints)]
        M.append(xi)
        m.append(v)
    return M,m
starts,ends = refine_data2(place, people, rpoints)
M,m = get_Mm(people_paths, ends, distance, rpoints)

In [27]:
# 预测每个民众达到的避难所

In [28]:
# 分割数据
from sklearn.model_selection import train_test_split
M_train, M_test, m_train, m_test = train_test_split(M,m,test_size=0.3,random_state=0)
# 数据缩放预处理
from sklearn.preprocessing import StandardScaler # Normalizer
scaler = StandardScaler()
scaler.fit(M_train)
M_train = scaler.transform(M_train)
M_test = scaler.transform(M_test)
# 选择模型(随机森林)
from sklearn.ensemble import RandomForestClassifier
rf2 = RandomForestClassifier(n_estimators=12)
#训练模型
rf2 = rf2.fit(M_train, m_train,)
# 在测试集上进行预测
m_pred = rf2.predict(M_test)
# 模型评估
from sklearn.metrics import accuracy_score
print('测试集的准确率为：{:.2f}%'.format(100*accuracy_score(m_test, m_pred)))  

测试集的准确率为：97.92%
